In [894]:
import connection as cn
import pandas as pd

from sqlalchemy import create_engine
from datetime import datetime

In [895]:
def highlight_odd_rows(row):
    # Check if the row index is odd
    if row.to  == 'central' :
        # Return a list of style strings for each cell in the row
        return ['background-color: #6E63F7'] * len(row)
    else:
        # Return an empty list for even rows (no styling)
        return [''] * len(row)


def highlight_rows(row):
    # Check if the row index is odd
    if row.to  == 'central' and row.froms == 'south':
        # Return a list of style strings for each cell in the row
        return ['background-color: #3379C4'] * len(row)
    
    elif row.to == 'chiangmai' and row.froms == 'central' : 
        return  ['background-color: #B83E3E'] * len(row)
    
    elif row.to == 'south' and row.froms == 'central': 
        return ['background-color: #033969'] * len(row)
    
    elif row.to == 'central' and row.froms == 'chiangmai':
        return ['background-color: #6F0000'] * len(row)

    else:
        # Return an empty list for even rows (no styling)
        return [''] * len(row)


In [896]:
engine = create_engine(f"postgresql+psycopg2://{cn.user}:{cn.password}@{cn.localhost}/{cn.database}",echo=False)

In [897]:
booking = pd.read_sql("select * from booking", engine)
region = pd.read_sql("select * from region",engine)
trailer = pd.read_sql("select * from trailer",engine)
destination = pd.read_sql("select * from destination",engine)
province = pd.read_sql("select * from province_entity",engine)
detail = pd.read_sql("select * from detail",engine).rename(columns={"id": "detail_id"})
user = pd.read_sql("select * from users",engine)
vehicle = pd.read_sql("select * from vehicle",engine)
current_date = datetime.now().date()

In [898]:
merged_df = booking.merge(region, left_on='fromId', right_on='id').merge(region, left_on='toId', right_on="id").drop(['id_y','id','fromId','toId'],axis=1).merge(trailer, left_on="trailerId", right_on="id").drop(['createDate_y','actExpire','id','trailerId'],axis=1).rename(columns={"region_x": "from","region_y": "to", "createDate_x": "createDate","registration":"trailer_registration"})

In [899]:
merged_df.createDate = merged_df.createDate.dt.date
merged_df.departureDate = merged_df.departureDate.dt.date
merged_df.arriveDate = merged_df.arriveDate.dt.date


In [900]:
merged_df = merged_df.astype({"createDate": "datetime64[ns]", 
                              "departureDate": "datetime64[ns]",
                              "arriveDate": "datetime64[ns]",
                              "status":"category",
                              "capacity":"Int8",
                              "from": "category",
                              "to":"category",
                              "deposit": "Int16",
                              "trailer_registration": "category"
                              }).rename(columns={"id_x": "booking_id"})



In [901]:

get_route  = (destination.merge(booking,left_on='booking', right_on="id").merge(province, left_on="province", right_on='id')).reset_index().groupby('booking').apply(lambda x: ','.join(x.province_y)).reset_index().rename(columns={0: 'route'});


C:\Users\User\AppData\Local\Temp\ipykernel_24992\731876730.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  get_route  = (destination.merge(booking,left_on='booking', right_on="id").merge(province, left_on="province", right_on='id')).reset_index().groupby('booking').apply(lambda x: ','.join(x.province_y)).reset_index().rename(columns={0: 'route'});


In [902]:
merged_df = merged_df.merge(get_route, left_on="booking_id", right_on='booking').sort_values('departureDate',ascending=False).drop(columns=["createDate"],axis=1)


In [903]:

detail = detail.merge(merged_df, left_on="bookingId", right_on="booking_id").drop(columns=['bookingId','deposit','booking']).merge(vehicle, left_on="vehicleId", right_on="id").drop(columns=["ownerId",'id',"vehicleId",'booking_id']).merge(
    user,left_on='userId', right_on="id").drop(columns=['createDate','age','userId','id']).merge(province,left_on="pickId",right_on="id").drop(columns=['id']).rename(columns={"province": "pick"}).merge(province, left_on="dropId",right_on='id').drop(columns=['id','pickId','dropId','capacity']).rename(columns={"province":"drop"})



In [904]:
details = detail.rename(columns={"from": "froms"}).sort_values('departureDate', ascending=False).drop(columns=["arriveDate","route",'sex']).query('status == "pending"').rename(columns={"tel":"telephone Number"})
# detail

In [905]:
bookings  = merged_df.query("status =='pending' | status =='depart'").sort_values(["departureDate"], ascending=True).assign(isChiangmai = lambda x: x.route.str.contains('chiangmai')).drop(columns=['booking'])

In [906]:
north = bookings.query("route.str.contains('chiangrai|chiangmai', regex=True,case=False)").sort_values(['departureDate'], ascending=True).set_index("booking_id").drop(columns=['route'])
north.departureDate = north.departureDate.dt.strftime('%d-%m-%Y')
north.arriveDate = north.arriveDate.dt.strftime('%d-%m-%Y')
isan = bookings.query('trailer_registration ==  "outsource_ta"').drop(columns=['isChiangmai','route']).set_index('booking_id')

# chiangmai = north.query('trailer_registration == "outsource_petch"')

# chiangrai = north.query('trailer_registration == "outsource_one"')



In [907]:
north.style.apply(highlight_odd_rows,axis=1)

,departureDate,arriveDate,capacity,status,deposit,from,to,trailer_registration,isChiangmai
booking_id,,,,,,,,,
51,16-09-2025,17-09-2025,0,pending,500,central,chiangmai,outsource_petch,True
52,19-09-2025,20-09-2025,0,pending,0,central,chiangmai,outsource_petch,True
53,23-09-2025,24-09-2025,0,pending,0,central,chiangmai,outsource_petch,True
54,26-09-2025,27-09-2025,0,pending,0,central,chiangmai,outsource_petch,True
55,30-09-2025,01-10-2025,0,pending,0,central,chiangmai,outsource_petch,True


In [908]:
south = bookings.query("route.str.contains('hatyai|phuket', regex=True,case=False)").sort_values(['departureDate']).drop(columns=['isChiangmai','route']).set_index("booking_id")
south.departureDate = south.departureDate.dt.strftime('%d-%m-%Y')
south.arriveDate = south.arriveDate.dt.strftime('%d-%m-%Y')
south.style.apply(highlight_odd_rows,axis=1)


,departureDate,arriveDate,capacity,status,deposit,from,to,trailer_registration
booking_id,,,,,,,,
40,16-09-2025,18-09-2025,7,pending,3000,central,south,722735
45,18-09-2025,20-09-2025,3,pending,1500,south,central,722735
41,22-09-2025,24-09-2025,2,pending,1000,central,south,722735
46,24-09-2025,26-09-2025,1,pending,500,south,central,722735
42,28-09-2025,30-09-2025,6,pending,3000,central,south,722735
47,30-09-2025,02-10-2025,0,pending,0,south,central,722735
43,03-10-2025,05-10-2025,0,pending,0,central,south,722735
58,05-10-2025,07-10-2025,4,pending,2000,south,central,722735


In [909]:
pending = details.drop(columns=['createDate_x','createDate_y']).set_index('detail_id').sort_values(['departureDate',"trailer_registration","name"])
pending.departureDate = pending.departureDate.dt.strftime('%d-%m-%Y')

pending = pending.reindex(columns=['departureDate',"pick",'drop','name','lastName','telephone Number','brand','registration','model','froms','to','trailer_registration','remark'])
pending.style.apply(highlight_rows, axis=1)

,departureDate,pick,drop,name,lastName,telephone Number,brand,registration,model,froms,to,trailer_registration,remark
detail_id,,,,,,,,,,,,,
98,16-09-2025,bangkok,hatyai,กิจชัย,หอพิสุทธิสาร,0867916666,toyota,ขน7171,land cruiser,central,south,722735,ผู้รับ วัชรพงศ์​ เรืองฤทธิ์ 0874619898
80,16-09-2025,bangkok,hatyai,ธีรุตม์,ปิตานุพงศ์,0973454770,bmw,6ขก2827,f30,central,south,722735,None
112,16-09-2025,bangkok,hatyai,นาอีม,หะสาเมาะ,0865993608,mitsubishi,กท5585,pajero,central,south,722735,None
95,16-09-2025,bangkok,phuket,มนัสนันท์,คอนเนลลี่,0909717143,suzuki,ขว421,carry,central,south,722735,ยอดค้างชำระ 6500 บาท พี่ต้าส่งต่อ
91,16-09-2025,bangkok,surat,สุกัญญา,สามสาลี,0971459695,honda,สฮ272,city,central,south,722735,รับรถที่ ตลาดโคอ๊อฟ
113,16-09-2025,bangkok,phuket,สุวภัทร,ส,0815844454,mini,6ขท7360,cooper,central,south,722735,None
115,16-09-2025,bangkok,surat,อารีรัตน์,ส,0849309353,chevrolet,ขข4202,zafira,central,south,722735,ส่งตรงข้ามสนามบินนครศรี *รถถึงแยกไชยาให้ติดต่อ คุณปอนด์
114,18-09-2025,phatthalung,bangkok,ชวินท์,จันทรมงคล,0931456529,benz,กต7058,190e,south,central,722735,จุดขึ้นรถ สี่แยกเอเซีย จังหวัดพัทลุง
110,18-09-2025,trang,ratchaburi,ธนกฤต,จันทรังษี,0899086133,toyota,ศฎ4930,hilux,south,central,722735,ลงวังมะนาว ผู้รับนายวีระพล ว่องวรพรกุล 0618744345


In [910]:
depart = detail.drop(columns=['createDate_x','route','createDate_y','sex']).query('departureDate.dt.date <= @current_date and status == "depart"').set_index('detail_id').sort_values('drop')
# depart.reindex(columns=['departureDate', "arriveDate","pick",'drop','name','lastName','tel','brand','registration','model','from','to','trailer_registration','remark'])

In [911]:
south_date = current_date - pd.Timedelta(days=1)
arrived = detail.query('status == "arrived" & arriveDate.dt.date == @current_date').drop(columns=['route','departureDate','sex','createDate_x','createDate_y']).set_index('detail_id')
arrived.reindex(columns=['arriveDate',"pick",'drop','name','lastName','tel','brand','registration','model','from','to','trailer_registration','remark'])



,arriveDate,pick,drop,name,lastName,tel,brand,registration,model,from,to,trailer_registration,remark
detail_id,,,,,,,,,,,,,
87,2025-09-14,phuket,bangkok,พิเชษฐ,เฉียงสระน้อย,0908003200,isuzu,ขฉ2066,dmax,south,central,722735,None
70,2025-09-14,phuket,bangkok,เขมกุลชญา,ศรีมนัส,0839818989,ford,6กส36,everett,south,central,722735,รับรถที่โกแจ้
75,2025-09-14,nakhonsi,bangkok,ขนิษฐา,กีรติภัทรกาญจน์,0862685544,volvo,กม6373,V50,south,central,722735,รับรถที่แยกสวนผัก ลูกค้ารีเควสขออยู่ด้านล่าง
90,2025-09-14,phuket,bangkok,พิชิต,เต็มสินสกุล,0870515678,nissan,1กถ926,sylphy,south,central,722735,083-6655366 คุณโต้ง เบอร์ผู้ส่ง
82,2025-09-14,phuket,bangkok,พี่,อุ๊,0842645324,volvo,ขค6937,v40,south,central,722735,เก็บเต็มจำนวน 3800
97,2025-09-14,chiangmai,bangkok,กิจชัย,หอพิสุทธิสาร,0867916666,toyota,ขน7171,land cruiser,chiangmai,central,outsource_petch,ส่งต่อหาดใหญ่วันที่ 16


In [912]:
today_booking = details.drop(columns=['createDate_y']).set_index('detail_id').rename(columns={"createDate_x": "createDate"}).query('createDate.dt.date == @current_date')
today_booking

,createDate,remark,departureDate,status,froms,to,trailer_registration,brand,registration,model,name,lastName,telephone Number,pick,drop
detail_id,,,,,,,,,,,,,,,


In [913]:
# user.set_index('id').sort_index().tail()
# user.query('tel == "0849309353"')
# user.query('name == "พิเชษฐ"').set_index('id')

In [914]:
# user_car =vehicle.merge(user, left_on="ownerId", right_on="id").set_index('id_x').drop( columns=['createDate_x','createDate_y','id_y','sex','age','ownerId','tel']).sort_index()
# user_car.tail()
# user_car.query('registration == "ขข4202"')

In [915]:
# chiangmai = detail.query("route.str.contains('chiangrai|chiangmai', regex=True,case=False)").assign(isChiangmai = lambda x: x.route.str.contains('chiangmai'))
# chiangmai.groupby(["isChiangmai", pd.Grouper(key='departureDate', freq='W-SUN')]).count()

In [916]:
# test = detail.groupby(['from', pd.Grouper(key='departureDate', freq='W-MON'),'to']).count()

# pending[(pending['departureDate'] == '2025-09-28 00:00:00') & (pending['brand'] == 'mini')].count()


In [917]:
# detail.query(f"status == 'depart' and departureDate.dt.date == @current_date").reset_index().drop(["index"],axis=1)
# weekly report
# detail.loc[:, ['createDate_x','departureDate','arriveDate',"from","to",'pick','drop']].groupby(['from','to']).resample('W-Mon',on="createDate_x").count() 

In [918]:
# user.sex.value_counts().plot.pie(y='sex',startangle=90)

In [919]:
# user.query('age > 18').age.value_counts().sort_index().plot.bar(color="green",title ="age")

In [920]:
# booking.drop(columns=['createDate']).sort_values('departureDate')

# detail.drop(columns=['remark', 'createDate_x', 'arriveDate','trailer_registration','route','createDate_y','from','to','registration','name','lastName','tel','departureDate','detail_id','status']).groupby(['pick','drop']).count()

In [921]:
# with pd.ExcelWriter('booking.xlsx', engine='xlsxwriter',datetime_format='yyyy-mm-dd') as writer:
#     chiangmai.to_excel(writer, sheet_name="chiangmai_booking",index=False, startrow=2)
#     chiangrai.to_excel(writer, sheet_name="chiangrai_booking", index=False, startrow=2)
#     south.to_excel(writer,sheet_name="south_booking",index=False, startrow=2)
#     isan.to_excel(writer,sheet_name="isan_booking",index=False, startrow=2)
#     pending.to_excel(writer,sheet_name="detail_booking",index=False, startrow=2)
#     depart.to_excel(writer,sheet_name="depart_booking",index=False, startrow=2)
#     arrived.to_excel(writer,sheet_name="arrived_booking",index=False, startrow=2)
#     writer.sheets["chiangmai_booking"].autofit()
#     writer.sheets["chiangrai_booking"].autofit()
#     writer.sheets["isan_booking"].autofit()
#     writer.sheets['south_booking'].autofit()
#     writer.sheets['detail_booking'].autofit()
#     writer.sheets['depart_booking'].autofit()
#     writer.sheets['arrived_booking'].autofit()